In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My_project/cleaneddataset.csv')


In [4]:
df.drop(['promptqtn_clean'], axis=1, inplace = True)


In [33]:
df.columns

Index(['text_clean', 'prompttext_clean', 'text_cnt_stopwords', 'text_cntpunct',
       'text_wordcnt', 'text_length', 'text_syllablecnt', 'text_ARI',
       'text_GFI', 'text_cntverb', 'text_cntadjctv', 'text_cntadvrb',
       'text_cntnoun', 'text_cntpronoun', 'text_cntspecificpos', 'text_bigram',
       'text_trigram', 'text_misspellcnt', 'text_unique',
       'cos_similarity_text_prompt', 'content', 'wording'],
      dtype='object')

In [34]:
order = ['prompttext_clean','text_clean','text_cnt_stopwords', 'text_cntpunct',
       'text_wordcnt', 'text_length', 'text_syllablecnt', 'text_ARI',
       'text_GFI', 'text_cntverb', 'text_cntadjctv', 'text_cntadvrb',
       'text_cntnoun', 'text_cntpronoun', 'text_cntspecificpos', 'text_bigram',
       'text_trigram', 'text_misspellcnt', 'text_unique',
       'cos_similarity_text_prompt', 'content', 'wording']

In [35]:
df = df[order]

In [36]:
df.columns

Index(['prompttext_clean', 'text_clean', 'text_cnt_stopwords', 'text_cntpunct',
       'text_wordcnt', 'text_length', 'text_syllablecnt', 'text_ARI',
       'text_GFI', 'text_cntverb', 'text_cntadjctv', 'text_cntadvrb',
       'text_cntnoun', 'text_cntpronoun', 'text_cntspecificpos', 'text_bigram',
       'text_trigram', 'text_misspellcnt', 'text_unique',
       'cos_similarity_text_prompt', 'content', 'wording'],
      dtype='object')

In [37]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/My_project/dataset.csv',index=False)

In [38]:
df.head(2)

,prompttext_clean,text_clean,text_cnt_stopwords,text_cntpunct,text_wordcnt,text_length,text_syllablecnt,text_ARI,text_GFI,text_cntverb,...,text_cntnoun,text_cntpronoun,text_cntspecificpos,text_bigram,text_trigram,text_misspellcnt,text_unique,cos_similarity_text_prompt,content,wording
0,chapter sequel already say must proceed consid...,element ideal tragedy arrange complex plan ano...,31,3,55,285,94,25.85,24.18,3,...,11,3,5,54,53,0,33,0.167416,-0.210614,-0.471415
1,chapter sequel already say must proceed consid...,three element ideal tragedy character not bad ...,12,6,26,143,44,12.95,15.02,3,...,6,2,2,25,24,1,25,0.172985,-0.970237,-0.417058


In [39]:
X = df.drop(['content','wording'],axis=1).values
Y = df[['content','wording']]

In [40]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y, random_state=100, test_size=0.2)

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [42]:
x_train_list = x_train.tolist()  # Convert to list (if needed)
x_train_list = [" ".join(map(str, doc)) for doc in x_train_list]  # Join words into strings

# Rest of your code (vectorization, model training, etc.)

x_train_vtrsd = vectorizer.fit_transform(x_train_list)



In [43]:
x_test_list = x_test.tolist()  # Convert to list (if needed)
x_test_list = [" ".join(map(str, doc)) for doc in x_test_list]

In [44]:
x_test_vtrsd = vectorizer.transform(x_test_list)


In [1]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [46]:
reg = MultiOutputRegressor(GradientBoostingRegressor) 

In [47]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [48]:
model = MultiOutputRegressor(GradientBoostingRegressor(random_state=0)).fit(x_train_vtrsd,y_train)

In [49]:
predi = model.predict(x_test_vtrsd)

In [50]:
r2_score(y_test,predi)

0.6225789233053038

In [51]:
mean_absolute_error(y_test,predi)

0.46720866003935213

In [52]:
mean_squared_error(y_test,predi)

0.35516610464657283

In [ ]:
import spacy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import string
from nltk.util import ngrams
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
!python3 -m spacy download en_core_web_md
nltk.download('punkt')
!pip3 install textstat
from textstat import gunning_fog
!pip install pyspellchecker
from spellchecker import SpellChecker

In [53]:
nlp = spacy.load("en_core_web_md")

def count_stopwords(text: str) -> int:
    stopwords_list = set(stopwords.words('english'))
    words = text.split()
    stopwords_count = sum(1 for word in words if word.lower() in stopwords_list)
    return stopwords_count

def count_punctuation(text: str) -> int:
    punctuation_set = set(string.punctuation)
    punctuation_count = sum(1 for char in text if char in punctuation_set)
    return punctuation_count


def collapse_dots(input):
    # Collapse sequential dots
    input = re.sub("\.+", ".", input)
    # Collapse dots separated by whitespaces
    all_collapsed = False
    while not all_collapsed:
        output = re.sub(r"\.(( )*)\.", ".", input)
        all_collapsed = input == output
        input = output
    return output

def clean_text(text:str, join_back=True):
  result = []
  sentences = sent_tokenize(text)
  sentences = [collapse_dots(sentence) for sentence in sentences]
  sentences = [sentence.strip() for sentence in sentences]

  for sentence in sentences:
    words = word_tokenize(sentence)

    words = [re.sub(r'\s+', ' ', word) for word in words]
    words = [re.sub(r'[^a-zA-Z\s]', '', word) for word in words]
    words = [re.sub(r'[^\w\s]', '', word) for word in words]
    words = [word.lower() for word in words]
    # words = [word for word in words if word not in stop_words]
    filtered_sentence = " ".join(words)
    result.append(filtered_sentence)

  return " ".join(result)

def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

def remove_extra_space(text:str):
  text = re.sub(r'\s+', ' ', text)
  text = text.strip()
  return text

def count_total_words(text: str) -> int:
    words = text.split()
    total_words = len(words)
    return total_words

def count_syllables(word):
    # Simple syllable counting (an approximation)
    vowels = "aeiouAEIOU"
    syllables = sum(1 for char in word if char in vowels)
    return syllables

def automated_readability_indexs(text):

    doc = nlp(text)
    # Calculate the number of words
    word_count = len(doc)

    # Calculate the number of sentences
    sentence_count = len(list(doc.sents))

    # Calculate the average number of letters per word
    letters_per_word = sum(len(word.text) for word in doc) / word_count

    # Calculate the average number of words per sentence
    words_per_sentence = word_count / sentence_count if sentence_count > 0 else 0

    # Calculate the ARI
    ari = 4.71 * letters_per_word + 0.5 * words_per_sentence - 21.43

    return ari

def gunning_fog_index(text):
  text = text
  return gunning_fog(text)


# Function to count verbs
def count_verbs(text):
    doc = nlp(text)
    verbs = [token for token in doc if token.pos_ == "VERB"]
    return len(verbs)

# Function to count adjectives
def count_adjectives(text):
    doc = nlp(text)
    adjectives = [token for token in doc if token.pos_ == "ADJ"]
    return len(adjectives)

# Function to count adverbs
def count_adverbs(text):
    doc = nlp(text)
    adverbs = [token for token in doc if token.pos_ == "ADV"]
    return len(adverbs)

# Function to count nouns
def count_nouns(text):
    doc = nlp(text)
    nouns = [token for token in doc if token.pos_ == "NOUN"]
    return len(nouns)

# Function to count pronouns
def count_pronouns(text):
    doc = nlp(text)
    pronouns = [token for token in doc if token.pos_ == "PRON"]
    return len(pronouns)

def count_specific_pos(text, pos_tag):
    # Count the number of words with a specific part-of-speech tag (e.g., "ADP" for prepositions)
    doc = nlp(text)
    pos_count = sum(1 for token in doc if token.pos_ == pos_tag)
    return pos_count

def count_bigrams(text: str) -> int:
    words = word_tokenize(text)
    bigrams = list(ngrams(words, 2))
    return len(bigrams)

def count_trigrams(text: str) -> int:
    words = word_tokenize(text)
    trigrams = list(ngrams(words, 3))
    return len(trigrams)

spell = SpellChecker()

def misspelled_cnt(text):
  spell = SpellChecker()
  misspelled = spell.unknown(text.split())
  count = len(misspelled)
  return count


def unique_count(text):
  doc = nlp(text)
  words = [token.text for token in doc]
  unique = len(set(words))
  return unique

# want to remove all the stopwords other than not , since if we remove 'not' it may affect the meaning of the sentence

stop_words = set(stopwords.words('english'))
stop_words.remove('not')


def remove_stopword(text):
    text_tokens = word_tokenize(text)
    text_tokens = [word for word in text_tokens if word not in stop_words]
    return ' '.join(text_tokens)

def cosine_similarity_text_prompt(text, prompt_text):
    # Tokenize and remove stopwords
    # Create TF-IDF vectors for text and prompt_text
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([text, prompt_text])

    # Calculate cosine similarity
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

    return cosine_sim[0][0]



In [ ]:
# def tokenization(prompt, summary):
#   features = [prompt,summary]
#   features = ["".join(map(str, doc)) for doc in features]
#   features = vectorizer.fit_transform(features) # Join words into strings
#   print(features)
#   return features

In [79]:
def prediction(prompt,summary):
  x = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  text_cntstpwrd = count_stopwords(summary)
  print(text_cntstpwrd)
  text_cnt_punct = count_punctuation(summary)
  summary = clean_text(summary)
  prompt = clean_text(prompt)
  summary = remove_extra_space(summary)
  prompt = remove_extra_space(prompt)
  text_wordcnt = count_total_words(summary)
  text_len = len(summary)
  text_syllablecnt = count_syllables(summary)
  text_ARI = automated_readability_indexs(summary)
  text_GFI = gunning_fog_index(summary)
  text_cntverb = count_verbs(summary)
  text_adjtv = count_adjectives(summary)
  text_cntadvrb = count_adverbs(summary)
  text_cntnoun = count_nouns(summary)
  text_cntpronoun = count_pronouns(summary)
  text_cntspecificpos = count_specific_pos(summary,'ADP')
  text_bigram = count_bigrams(summary)
  text_trigram = count_trigrams(summary)
  text_misspelt = misspelled_cnt(summary)
  text_unique = unique_count(summary)
  summary = remove_stopword(summary)
  prompt = remove_stopword(prompt)
  summary = lemmatize_text(summary)
  prompt = lemmatize_text(prompt)
  cosine_similarity = cosine_similarity_text_prompt(summary,prompt)

  x[0] = prompt
  x[1] = summary
  x[2] = text_cntstpwrd
  x[3] = text_cnt_punct
  x[4] = text_wordcnt
  x[5] = text_len
  x[6] = text_syllablecnt
  x[7] = text_ARI
  x[8] = text_GFI
  x[9] = text_cntverb
  x[10] = text_adjtv
  x[11] = text_cntadvrb
  x[12] = text_cntnoun
  x[13] = text_cntpronoun
  x[14] = text_cntspecificpos
  x[15] = text_bigram
  x[16] = text_trigram
  x[17] = text_misspelt
  x[18] = text_unique
  x[19] = cosine_similarity

  x_list = [" ".join(map(str, x))]
  x_vctrsd = vectorizer.transform(x_list)

  return model.predict(x_vctrsd)







In [86]:
df['wording'][1]

-0.417058297304168

In [83]:
prompt = '''Chapter 13
As the sequel to what has already been said, we must proceed to consider what the poet should aim at, and what he should avoid, in constructing his plots; and by what means the specific effect of Tragedy will be produced.
A perfect tragedy should, as we have seen, be arranged not on the simple but on the complex plan. It should, moreover, imitate actions which excite pity and fear, this being the distinctive mark of tragic imitation. It follows plainly, in the first place, that the change of fortune presented must not be the spectacle of a virtuous man brought from prosperity to adversity: for this moves neither pity nor fear; it merely shocks us. Nor, again, that of a bad man passing from adversity to prosperity: for nothing can be more alien to the spirit of Tragedy; it possesses no single tragic quality; it neither satisfies the moral sense nor calls forth pity or fear. Nor, again, should the downfall of the utter villain be exhibited. A plot of this kind would, doubtless, satisfy the moral sense, but it would inspire neither pity nor fear; for pity is aroused by unmerited misfortune, fear by the misfortune of a man like ourselves. Such an event, therefore, will be neither pitiful nor terrible. There remains, then, the character between these two extremes — that of a man who is not eminently good and just, yet whose misfortune is brought about not by vice or depravity, but by some error of judgement or frailty. He must be one who is highly renowned and prosperous — a personage like Oedipus, Thyestes, or other illustrious men of such families.
A well-constructed plot should, therefore, be single in its issue, rather than double as some maintain. The change of fortune should be not from bad to good, but, reversely, from good to bad. It should come about as the result not of vice, but of some great error or frailty, in a character either such as we have described, or better rather than worse. The practice of the stage bears out our view. At first the poets recounted any legend that came in their way. Now, the best tragedies are founded on the story of a few houses — on the fortunes of Alcmaeon, Oedipus, Orestes, Meleager, Thyestes, Telephus, and those others who have done or suffered something terrible. A tragedy, then, to be perfect according to the rules of art, should be of this construction. Hence they are in error who censure Euripides just because he follows this principle in his plays, many of which end unhappily. It is, as we have said, the right ending. The best proof is that on the stage and in dramatic competition, such plays, if well worked out, are the most tragic in effect; and Euripides, faulty though he may be in the general management of his subject, yet is felt to be the most tragic of the poets.
In the second rank comes the kind of tragedy which some place first. Like the Odyssey, it has a double thread of plot, and also an opposite catastrophe for the good and for the bad. It is accounted the best because of the weakness of the spectators; for the poet is guided in what he writes by the wishes of his audience. The pleasure, however, thence derived is not the true tragic pleasure. It is proper rather to Comedy, where those who, in the piece, are the deadliest enemies — like Orestes and Aegisthus — quit the stage as friends at the close, and no one slays or is slain.'''
summary = '''The three elements of an ideal tragedy are:  Having a character that isn't bad have misfortune befall them.,  Having no subplots, and ending in death.  '''
prediction(prompt,summary)

12


array([[-1.07638109, -1.11882929]])

In [60]:
df.columns

Index(['prompttext_clean', 'text_clean', 'text_cnt_stopwords', 'text_cntpunct',
       'text_wordcnt', 'text_length', 'text_syllablecnt', 'text_ARI',
       'text_GFI', 'text_cntverb', 'text_cntadjctv', 'text_cntadvrb',
       'text_cntnoun', 'text_cntpronoun', 'text_cntspecificpos', 'text_bigram',
       'text_trigram', 'text_misspellcnt', 'text_unique',
       'cos_similarity_text_prompt', 'content', 'wording'],
      dtype='object')

In [87]:
import joblib
# Save the model to a file
joblib.dump(model, '/content/drive/MyDrive/Colab Notebooks/My_project/ml_model.joblib')


['/content/drive/MyDrive/Colab Notebooks/My_project/ml_model.joblib']